get data

pollutants

In [ ]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta

# Website link/datasheet -> loaction of stations findable from here too: https://luftdaten.berlin.de/pollution/


def clean_station_data(df: pd.DataFrame, measurement: str) -> pd.DataFrame:
    """
    Cleans a station dataframe:
    - Drops first three metadata rows
    - Converts from wide to long format
    - Parses measurement time
    - Adds a 'measurement' column
    """
    # Drop first three rows
    df = df.iloc[3:].copy()
    
    # Rename columns: first column is 'time', others stay as station names
    df = df.rename(columns={df.columns[0]: 'time'})
    
    # Melt the dataframe to long format
    df_long = df.melt(id_vars='time', var_name='station', value_name='value')
    
    # Parse time column
    df_long['time'] = pd.to_datetime(df_long['time'], format='%d.%m.%Y %H:%M')
    
    # Add measurement column
    df_long['measurement'] = measurement

    # Convert values to numeric (in case there are missing or non-numeric entries)
    df_long['value'] = pd.to_numeric(df_long['value'], errors='coerce')
    

    
    return df_long


def download_and_clean(pollutant: str, start: datetime, end: datetime) -> pd.DataFrame:
    """
    Downloads and cleans data for a given pollutant from Berlin Luftdaten.
    """
    base_url = f"https://luftdaten.berlin.de/core/{pollutant}.csv?stationgroup=all&period=1h&timespan=custom&start[date]={{start_date}}&start[hour]={{start_hour}}&end[date]={{end_date}}&end[hour]={{end_hour}}"
    
    delta = timedelta(days=30)
    all_data = []

    current_start = start
    while current_start < end:
        current_end = min(current_start + delta, end)
        
        # Format dates
        start_date = current_start.strftime("%d.%m.%Y")
        start_hour = current_start.strftime("%H")
        end_date = current_end.strftime("%d.%m.%Y")
        end_hour = current_end.strftime("%H")
        
        # Build URL
        url = base_url.format(start_date=start_date, start_hour=start_hour,
                              end_date=end_date, end_hour=end_hour)
        
        print(f"Downloading {pollutant.upper()} data from {start_date} {start_hour} to {end_date} {end_hour}")
        
        # Download CSV
        response = requests.get(url)
        if response.status_code == 200:
            df = pd.read_csv(StringIO(response.text), sep=";")
            cleaned = clean_station_data(df, pollutant)
            all_data.append(cleaned)
        else:
            print(f"Error downloading {pollutant}: {response.status_code}")
        
        current_start = current_end + timedelta(hours=1)
    
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame(columns=['time', 'station', 'value', 'measurement'])


# Define period
start = datetime(2024, 1, 1, 0)
end = datetime(2024, 12, 31, 23)

# Download & clean each pollutant
pollutants = ['no2', 'pm10', 'o3']
dfs = [download_and_clean(p, start, end) for p in pollutants]

# Combine into one long dataframe
full_df = pd.concat(dfs, ignore_index=True)

print(full_df.head(20))


                  time      station  value measurement
0  2024-01-01 00:00:00  010 Wedding   16.0         no2
1  2024-01-01 01:00:00  010 Wedding   42.0         no2
2  2024-01-01 02:00:00  010 Wedding   42.0         no2
3  2024-01-01 03:00:00  010 Wedding   30.0         no2
4  2024-01-01 04:00:00  010 Wedding   23.0         no2
5  2024-01-01 05:00:00  010 Wedding   19.0         no2
6  2024-01-01 06:00:00  010 Wedding   19.0         no2
7  2024-01-01 07:00:00  010 Wedding   15.0         no2
8  2024-01-01 08:00:00  010 Wedding   12.0         no2
9  2024-01-01 09:00:00  010 Wedding   12.0         no2
10 2024-01-01 10:00:00  010 Wedding   10.0         no2
11 2024-01-01 11:00:00  010 Wedding    9.0         no2
12 2024-01-01 12:00:00  010 Wedding   10.0         no2
13 2024-01-01 13:00:00  010 Wedding   11.0         no2
14 2024-01-01 14:00:00  010 Wedding   10.0         no2
15 2024-01-01 15:00:00  010 Wedding   12.0         no2
16 2024-01-01 16:00:00  010 Wedding   15.0         no2
17 2024-01

In [4]:
full_df.to_csv("./data/2024-pollutants.csv")

weather

In [32]:
import pandas as pd
from wetterdienst.provider.dwd.observation import DwdObservationRequest
import datetime as dt

# REFERENCE! https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/
# API version 0.113.0
# Berlin station IDs
berlin_stations = ("00399", "00400", "00403", "00410", "00420", "00424", "00427", "00430", "00433")

# Map of parameters to dataset codes
parameters = [
    ("hourly", "temperature_air"),
    ("hourly", "pressure"),  # Air pressure
    ("hourly", "moisture"),  # Humidity
    ("hourly", "precipitation"),  # precipitation height
    ("hourly", "wind_synop")
]

request = DwdObservationRequest(
    parameters = parameters,
    start_date="2024-01-01",
    end_date="2024-12-31"
)
stations = request.filter_by_station_id(station_id=berlin_stations)
df = stations.values.all().df.drop_nulls()
df

cols = df.columns

In [36]:
df = pd.DataFrame(df)

df.columns = cols

df.to_csv("./data/2024-weather.csv")

traffic